# G-LiHT 

This notebook illustrates search, directly loading DEMs, and downloading DEMs and LPCs for local processing

| Dataset                      | Alias      | Type      | Start      | End        | Extent         | Source                                                                      |
| ---------------------------- | ---------- | --------- | ---------- | ---------- | -------------- | --------------------------------------------------------------------------- |
| NASA G-LiHT                  | gliht      | LiDAR     | 2011-07-28 | 2020-03-12 | US Territories | [NASA G-LiHT](https://gliht.gsfc.nasa.gov/)         

In [ ]:
import coincident
import geopandas as gpd
from shapely.geometry import box

## Search 

We'll search for data in Maryland

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/MD/shape.geojson"
)
aoi = aoi.simplify(0.01)
m = aoi.explore(color="black")

gf_gliht = coincident.search.search(
    dataset="gliht",
    intersects=aoi,
    datetime=["2017-07-31"],
)
print(f"Found {len(gf_gliht)} G-LiHT scenes")
gf_gliht.explore(m=m, color="magenta")

In [ ]:
# Subset a particular scene
gf_gliht = gf_gliht.iloc[[0]]
c = gf_gliht.geometry.centroid
mini_aoi = gpd.GeoDataFrame(
    geometry=[box(c.x - 0.0045, c.y - 0.0045, c.x + 0.0045, c.y + 0.0045)],
    crs="EPSG:4326",
)

In [ ]:
m = gf_gliht.explore(color="black")
mini_aoi.explore(color="magenta", m=m)

## Get Data

NASA G-LiHT has many different provided gridded datasets. The following collections below are the current datasets supported by `coincident`.

```python
    - 'ortho': Orthorectified aerial imagery
    - 'chm': Canopy height model
    - 'dsm': Digital surface model
    - 'dtm': Digital terrain model
    - 'hyperspectral_ancillary': Ancillary HSI data
    - 'radiance': Hyperspectral aradiance
    - 'reflectance': Hyperspectral surface reflectance
    - 'hyperspectral_vegetation': HSI-derived veg indices
```

```{note}
Not all G-LiHT flights will contain every single product listed. For example, a flight may have 'dsm' data but not 'ortho' data.
```

### Stream 

First we'll show how to stream gridded DEMs directly into Xarray

In [ ]:
# we'll need the dataset id from our initial search
gf_gliht.id.item()

```{important}
Unlike the G-LiHT search, you will need NASA Earthdata credentials (aka EarthData Login (EDL)) to read in and download the gridded datasets from G-LiHT. This requires creating an [EDL Token](https://urs.earthdata.nasa.gov/documentation/for_users/user_token) and making sure you've set the environment variable `EARTHDATA_TOKEN=xxxxx`
```

In [ ]:
%%time
ds = coincident.io.xarray.load_gliht_raster(
    aoi=mini_aoi,
    dataset_id=gf_gliht.id.item(),
    product="chm",
)
ds

In [ ]:
# Convert to in-memory DataArray
da_chm = ds["CHM"].squeeze().load()

In [ ]:
# Plot
da_chm.plot.imshow(cmap="Greens");

### Download 

```{note}
Since there are so many gridded products for G-LiHT, you can specify multiple products to download in one call. This download will automatically skip any products that aren't available for the specific flight among the current datasets supported by `coincident`
```

In [ ]:
coincident.io.download.download_gliht_raster(
    aoi=mini_aoi,
    dataset_id=gf_gliht.id.item(),
    products=["chm", "dsm", "dtm"],
    output_dir="/tmp",
)